<a href="https://colab.research.google.com/github/joymaurya/Fine-Tune-Name-Entity-Recognition/blob/main/NER_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

In [ ]:
data=load_dataset('conll2003')

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
data['train'].features['ner_tags'].feature.names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [ ]:
from transformers import AutoTokenizer

In [ ]:
checkpoint='distilbert-base-cased'

In [ ]:
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
label_names=data['train'].features['ner_tags'].feature.names

In [ ]:
label2lb={1:2,3:4,5:6,7:8}

In [ ]:
label2lb.keys()

dict_keys([1, 3, 5, 7])

In [ ]:
def tokenize_data(batch):
  tokenized_string=tokenizer(batch['tokens'],is_split_into_words=True,truncation=True)
  tokens=[]
  i=None
  for j in tokenized_string.word_ids():
      if j==None:
        tokens.append(-100)
        i=j
      elif i!=j:
        tokens.append(batch['ner_tags'][j])
        i=j
      elif i==j:
        if batch['ner_tags'][j] in label2lb.keys():
          tokens.append(batch['ner_tags'][j]+1)
        else:
          tokens.append(batch['ner_tags'][j])
  tokenized_string['labels']=tokens
  return tokenized_string

In [ ]:
tokenized_data=data.map(tokenize_data,remove_columns=data['train'].column_names)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForTokenClassification

In [ ]:
data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=0aa2845e3bc7a690d74bf62d6cf3704a85f7dd066be0adc2fc59d5d9427ab361
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
from datasets import load_metric

In [ ]:
metrics=load_metric('seqeval')

<ipython-input-23-ba872ffe9888>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metrics=load_metric('seqeval')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
import numpy as np

In [ ]:
def compute_acc(logits_and_labels):
  logits,labels=logits_and_labels
  predict=np.argmax(logits,axis=-1)
  actual_labels=[[label_names[t] for t in sen if t!=-100] for sen in labels]
  predicted_labels=[[ label_names[b] for a,b in zip(n,m) if b!=-100] for n,m in zip(predict,labels)]
  the_metrics=metrics.compute(predictions=predicted_labels,references=actual_labels)
  return {
      'precision':the_metrics['overall_precision'],
      'recall':the_metrics['overall_recall'],
      'f1':the_metrics['overall_f1'],
      'accuracy':the_metrics['overall_accuracy']
  }

In [ ]:
id2label={k:v for k,v in enumerate(label_names)}

In [ ]:
label2id={v:k for k,v in enumerate(label_names)}

In [ ]:
from transformers import AutoModelForTokenClassification

In [ ]:
model=AutoModelForTokenClassification.from_pretrained(checkpoint,id2label=id2label,label2id=label2id)

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments

In [ ]:
training_arg=TrainingArguments(
    'my_ner_model',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01
    )

In [ ]:
from transformers import Trainer

In [ ]:
trainer=Trainer(
    model,
    args=training_arg,
    train_dataset=tokenized_data['train'],
    eval_dataset=tokenized_data['validation'],
    data_collator=data_collator,
    compute_metrics=compute_acc,
    tokenizer=tokenizer
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.093100,0.086252,1.000000,1.000000,1.000000,1.000000
2,0.044600,0.074140,1.000000,1.000000,1.000000,1.000000
3,0.033000,0.069415,1.000000,1.000000,1.000000,1.000000


TrainOutput(global_step=5268, training_loss=0.07991688189944628, metrics={'train_runtime': 318.0043, 'train_samples_per_second': 132.46, 'train_steps_per_second': 16.566, 'total_flos': 460336113849150.0, 'train_loss': 0.07991688189944628, 'epoch': 3.0})

In [ ]:
trainer.save_model('100_model')

In [ ]:
from transformers import pipeline

In [ ]:
ner=pipeline('token-classification',model='100_model',aggregation_strategy='simple',device=0)

In [ ]:
s='Bill Gates was CEO of of Microsoft in Seattle,Washington'

In [ ]:
ner(s)

[{'entity_group': 'PER',
  'score': 0.9992288,
  'word': 'Bill Gates',
  'start': 0,
  'end': 10},
 {'entity_group': 'ORG',
  'score': 0.9990212,
  'word': 'Microsoft',
  'start': 25,
  'end': 34},
 {'entity_group': 'LOC',
  'score': 0.99846894,
  'word': 'Seattle',
  'start': 38,
  'end': 45},
 {'entity_group': 'LOC',
  'score': 0.99653155,
  'word': 'Washington',
  'start': 46,
  'end': 56}]